In [11]:
import cv2
import numpy as np
import pandas as pd
import os
from skimage.feature import hog, local_binary_pattern
from sklearn.decomposition import PCA
from concurrent.futures import ProcessPoolExecutor
from sklearn.model_selection import train_test_split

In [6]:
train_dir = 'Dataset/Train'
test_dir = 'Dataset/Test'

def get_data(dir):
    images = []
    labels = []
    
    for cat in os.listdir(dir):
        for img_name in os.listdir(os.path.join(dir, cat)):
            img = cv2.imread(os.path.join(dir,cat,img_name), cv2.IMREAD_GRAYSCALE)
            images.append(img)
            labels.append(cat)
    
    return np.array(images), np.array(labels)

train_images, train_labels = get_data(train_dir)
test_images, test_labels = get_data(test_dir)

all_images = np.concatenate((train_images, test_images))
all_labels = np.concatenate((train_labels, test_labels))

print(all_images.shape, all_labels.shape)


(35887, 48, 48) (35887,)


In [9]:
print(np.unique(all_labels))
# describe the labels
print(pd.Series(all_labels).value_counts())

['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']
happy       8989
neutral     6198
sad         6077
fear        5121
angry       4953
surprise    4002
disgust      547
Name: count, dtype: int64


In [12]:
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

print(f"Train data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

Train data shape: (28709, 48, 48)
Test data shape: (7178, 48, 48)


In [ ]:
import cv2
import numpy as np
from skimage.feature import hog

# Extract HOG features
def extract_hog_features(image):
    return hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')

# Extract ORB features
def extract_orb_features(image, max_features=128):
    orb = cv2.ORB_create()
    _, descriptors = orb.detectAndCompute(image, None)
    if descriptors is None:
        return np.zeros(max_features * 32)
    if descriptors.shape[0] < max_features:
        padding = np.zeros((max_features - descriptors.shape[0], descriptors.shape[1]))
        descriptors = np.vstack((descriptors, padding))
    return descriptors[:max_features].flatten()

# Extract histogram features (for grayscale images)
def extract_histogram(image):
    # Compute the histogram with 256 bins (grayscale)
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    # Normalize the histogram
    hist = cv2.normalize(hist, hist).flatten()
    return hist

# Combine selected features
def extract_combined_features(image):
    hog_features = extract_hog_features(image)
    orb_features = extract_orb_features(image)
    histogram = extract_histogram(image)
    
    return np.concatenate([hog_features, orb_features, histogram])
